In [50]:
from keras.models import Sequential
from keras.layers import Dense
import numpy
numpy.random.seed(7)

In [51]:
# load pima indians dataset
dataset = numpy.loadtxt("pima-indians-diabetes.data.csv", delimiter=",")
# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]

Load data and slipt them to train/test part

In [52]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

### Create very simple model with only 8 input and 4 output 

In [53]:
# create model
model = Sequential()
model.add(Dense(4, input_dim=8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [54]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [55]:
# Fit the model
model.fit(X_train, y_train, epochs=50, batch_size=10)

Epoch 1/50
514/514 [==============================] - 0s 789us/step - loss: 1.4006 - acc: 0.5798
Epoch 2/50
514/514 [==============================] - 0s 189us/step - loss: 1.0898 - acc: 0.6109
Epoch 3/50
514/514 [==============================] - 0s 196us/step - loss: 0.9254 - acc: 0.6420
Epoch 4/50
514/514 [==============================] - ETA: 0s - loss: 0.8344 - acc: 0.670 - 0s 186us/step - loss: 0.8294 - acc: 0.6537
Epoch 5/50
514/514 [==============================] - 0s 224us/step - loss: 0.7877 - acc: 0.6537
Epoch 6/50
514/514 [==============================] - 0s 223us/step - loss: 0.7213 - acc: 0.6615
Epoch 7/50
514/514 [==============================] - 0s 193us/step - loss: 0.7029 - acc: 0.6634
Epoch 8/50
514/514 [==============================] - 0s 200us/step - loss: 0.6867 - acc: 0.6537
Epoch 9/50
514/514 [==============================] - 0s 184us/step - loss: 0.6731 - acc: 0.6634
Epoch 10/50
514/514 [==============================] - 0s 180us/step - loss: 0.6614 - acc

In [56]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 4)                 36        
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 5         
Total params: 41
Trainable params: 41
Non-trainable params: 0
_________________________________________________________________
None


In [57]:
scores = model.evaluate(X_test, y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

254/254 [==============================] - 0s 314us/step

acc: 69.29%


In [58]:
model.predict_classes(X_test[0].reshape(1,8))

array([[0]], dtype=int32)

In [59]:
for w in model.get_weights():
    print(w.shape)
    print(w)

(8, 4)
[[ 0.28950396  0.2662049  -0.10448706  0.04231657]
 [ 0.1396324  -0.12853253 -0.70365155  0.035602  ]
 [-0.5436991  -0.54288614 -0.12432242 -0.5320099 ]
 [ 0.3639847  -0.4966212  -0.04041024  0.53395003]
 [-0.0057458  -0.0406636  -0.49122    -0.01574758]
 [ 0.02793237 -0.11602473  0.12300175 -0.2505481 ]
 [-0.01053008 -0.06974089  0.00997893  0.23737665]
 [ 0.13264194  0.22553486  0.2507374   0.37397593]]
(4,)
[-0.2020333   0.         -0.1813703   0.16433579]
(4, 1)
[[ 0.27707627]
 [-0.033831  ]
 [ 0.77249765]
 [-0.6942656 ]]
(1,)
[-0.7259764]


## We factorization matrix W_1 to 3 compement matrix: U, s, VT

In [60]:
from numpy import array
from numpy import diag
from numpy import dot
from numpy import zeros
import numpy as np

w_1 = model.get_weights()[0]
U, s, VT = np.linalg.svd(w_1, full_matrices=True)
# create m x n Sigma matrix
Sigma = zeros((w_1.shape[0], w_1.shape[1]))
# populate Sigma with n x n diagonal matrix
Sigma[:w_1.shape[1], :w_1.shape[1]] = diag(s)
# reconstruct matrix
B = U.dot(Sigma.dot(VT))

In [61]:
B

array([[ 0.28950395,  0.2662049 , -0.10448707,  0.04231658],
       [ 0.13963241, -0.12853254, -0.70365156,  0.03560201],
       [-0.54369904, -0.54288613, -0.12432241, -0.53200989],
       [ 0.3639847 , -0.49662118, -0.04041027,  0.53395004],
       [-0.00574578, -0.0406636 , -0.49122001, -0.01574757],
       [ 0.02793236, -0.11602473,  0.12300176, -0.25054809],
       [-0.01053008, -0.06974089,  0.00997892,  0.23737664],
       [ 0.13264193,  0.22553486,  0.2507374 ,  0.37397593]])

In [62]:
print("U: ", U.shape)
print("Sigma: ", Sigma.shape)
print("VT: ", VT.shape)

U:  (8, 8)
Sigma:  (8, 4)
VT:  (4, 4)


In [63]:
model2 = Sequential()
model2.add(Dense(8, input_dim=8, activation='relu'))
model2.add(Dense(4, input_dim=8, activation='relu'))
model2.add(Dense(4, input_dim=4, activation='relu'))
model2.add(Dense(1, activation='sigmoid'))

In [64]:
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_11 (Dense)             (None, 8)                 72        
_________________________________________________________________
dense_12 (Dense)             (None, 4)                 36        
_________________________________________________________________
dense_13 (Dense)             (None, 4)                 20        
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 5         
Total params: 133
Trainable params: 133
Non-trainable params: 0
_________________________________________________________________


In [65]:
for wwww in model.get_weights():
    print (wwww.shape)

(8, 4)
(4,)
(4, 1)
(1,)


In [66]:
for wwww in model2.get_weights():
    print (wwww.shape)

(8, 8)
(8,)
(8, 4)
(4,)
(4, 4)
(4,)
(4, 1)
(1,)


In [67]:
weights = [U,
           model2.get_weights()[1], 
           Sigma, 
           model2.get_weights()[3], 
           VT, 
           model2.get_weights()[5],
           model.get_weights()[2],
           model.get_weights()[3]]
for w in weights:
    print(w.shape)

(8, 8)
(8,)
(8, 4)
(4,)
(4, 4)
(4,)
(4, 1)
(1,)


In [68]:
model2.set_weights(weights)
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

## Compare 2 models

### Model 1:

In [69]:
scores = model.evaluate(X_test, y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

254/254 [==============================] - 0s 453us/step

acc: 69.29%


In [70]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
confusion_matrix(y_test, model.predict_classes(X_test))

array([[161,   7],
       [ 71,  15]])

In [71]:
precision_recall_fscore_support(y_test, model.predict_classes(X_test), average='macro')

(0.6878918495297806, 0.5663759689922481, 0.5413888888888889, None)

## Model 2:

In [72]:
scores2 = model2.evaluate(X_test, y_test)
print("\n%s: %.2f%%" % (model2.metrics_names[1], scores2[1]*100))

254/254 [==============================] - 0s 481us/step

acc: 66.54%


In [73]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
confusion_matrix(y_test, model2.predict_classes(X_test))

array([[167,   1],
       [ 84,   2]])

In [74]:
precision_recall_fscore_support(y_test, model2.predict_classes(X_test), average='macro')

(0.6660026560424966, 0.5086517165005537, 0.4210399292054383, None)